In [3]:
#Importing the required libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#Importing the csv files into dataframes (both train and test)

dftrain = pd.read_csv("/content/drive/MyDrive/WOC/Polynomial_train.csv", header=[0])
del dftrain['Unnamed: 0']
dftest = pd.read_csv("/content/drive/MyDrive/WOC/Polynomial_test.csv", header=[0])
del dftest['Unnamed: 0']

In [5]:
#Defining X and Y

def importing(dataframe):
    X = dataframe.iloc[:,1:].transpose().to_numpy()
    features = X.shape[0]
    examples = X.shape[1]
    Y = dataframe.iloc[:,0]
    return X, Y, features, examples

Xtrain, Ytrain, trainfeatures, trainexamples = importing(dftrain)
Xtest, Ytest, testfeatures, testexamples = importing(dftest)

In [6]:
#Appending row with all ones and polynomial features to Xtrain

def polynomialfeatures(X, examples):
  X0 = np.ones((1, examples))
  X1 = np.square(X[0,:]).reshape(1,examples)
  X2 = np.square(X[1,:]).reshape(1,examples)
  X3 = np.square(X[2,:]).reshape(1,examples)
  XX1 = np.multiply(X1, X2)
  XX2 = np.multiply(X2, X3)
  XX3 = np.multiply(X3, X1)
  X = np.concatenate((X0, X, X1, X2, X3, XX1, XX2, XX3))
  return X

Xtrain = polynomialfeatures(Xtrain, trainexamples)
Xtest = polynomialfeatures(Xtest, testexamples)

In [7]:
#Applying Mean Normalization on the training data
def normalization(X):
  X = pd.DataFrame(X)
  print(X)
  for i in range(1,trainfeatures + 1):
    rowmin = X.iloc[i,:].min()
    rowmax = X.iloc[i,:].max()
    rowmean = X.iloc[i,:].mean()
    valrng = rowmax - rowmin
    X.iloc[i,:] -= rowmin
    X.iloc[i,:] -= rowmean
    X.iloc[i,:] /= valrng
  X = X.to_numpy()
  return X
Xtrain = normalization(Xtrain)
Xtest = normalization(Xtest)

          0             1              2          3             4      \
0  1.000000e+00  1.000000e+00       1.000000   1.000000  1.000000e+00   
1 -9.866550e+00 -3.761945e+00      -1.836218  -1.923651  3.583565e-01   
2 -5.836019e+00 -8.756162e+00      -1.846066   3.892972  6.703989e+00   
3  3.079729e+02  1.363984e+02    -310.724813   1.755447 -3.435826e+02   
4  9.734881e+01  1.415223e+01       3.371695   3.700432  1.284194e-01   
5  3.405912e+01  7.667037e+01       3.407958  15.155232  4.494346e+01   
6  9.484732e+04  1.860452e+04   96549.909713   3.081594  1.180490e+05   
7  3.315615e+03  1.085057e+03      11.490595  56.080900  5.771612e+00   
8  3.230417e+06  1.426415e+06  329038.061700  46.702270  5.305531e+06   
9  9.233274e+06  2.632955e+05  325536.824235  11.403227  1.515978e+04   

          5             6             7             8             9      ...  \
0  1.000000e+00  1.000000e+00  1.000000e+00  1.000000e+00  1.000000e+00  ...   
1 -6.995365e+00  9.350915e+00  6.725

In [8]:
theta = np.zeros((10, 1))
regtheta = np.concatenate([np.zeros((1,1)), theta[1:,]])
lamda = 100

def model(X, Y, theta, learningrate, iteration):
  costlist = []
  for i in range(iteration):
    y_pred = np.dot(X.T, theta)
    cost = (1/(2*trainexamples))*(np.sum(np.square(y_pred - Y))) + lamda*np.sum(np.square(regtheta))
    pdcost = (1/trainexamples)*np.dot(X, y_pred - Y)
    temptheta = theta*(1-learningrate*lamda/trainexamples) - learningrate*pdcost
    theta = temptheta
    costlist.append(cost)
  return theta, y_pred, costlist

In [ ]:
#Extracting theta, prediction and cost, and plotting the cost versus iterations graph

theta, y_pred, costlist = model(Xtrain, Ytrain, theta, 0.999, 1000)
plt.plot([i for i in range(1000)], costlist)
plt.show()

In [ ]:
#Finding the error

Yfinal = Ytest.to_numpy().reshape(testexamples, 1)

Ypredfinal = np.dot(Xtest.T, theta)
rmserror_quadratic = np.sqrt((np.sum(np.square(Yfinal - Ypredfinal)))/testexamples)

print(rmserror_quadratic)